<h1> PAC BOUNDS-NAIVE APPROACH </h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [2]:
def normalise(array):
    maximum=max(array)
    minimum=min(array)
    for i in range(len(array)):
        array[i]=(array[i]-minimum)/(maximum-minimum)
    return array

In [3]:
def epsilon_greedy(k,eps,iterations,mu='random'):
    
    #k=number of arms
    #eps=exploration fraction-> algorithm will explore with a probability of eps and exploit with a probability of (1-eps)
    #iterations= number of pulls of the arms
    #mu=an array of length k, which holds the true expected value of each arm
    #mu="random"-> the expected value is sampled from a normal distribution with standard deviation 1 (default)
    #mu=user defined-> the user sends the true expected values of each arm
    
    if mu=="random":
        mu=np.random.normal(0,1,k) #now mu is an array containing the true expected values of each arm
    if len(mu)!=k: #case wwhen mu is user defined
        print("The length of the entered array of true expected values does not match with the number of arms entered \n")
        return
    
    q=np.zeros(k) #the array of the estimated expected values of each arm
    pulls=0 #number of pulls at current iteration
    arm_pull_number=np.zeros(k) #stores the number of times each arm has been pulled till the current iteration
    reward=0 #the reward earned at the current iteration
    a=0 #the arm pulled at the current iteration
    total_reward=0 #the total rewards won till the current iteration
    avg_reward=np.zeros(iterations) #the average reward per iteration till the current iteration
    
    for step in range(iterations):
        p=np.random.rand() #randomly generates a number between 0 and 1
        
        if eps==0 and step==0: #eps value indicated exploitation but since the steps are also zero there is no knowledge to exploit-randomly select any arm
            a=np.random.choice(k) #action is chosen
        
        elif p<eps: #case of exploration
            a=np.random.choice(k) #action is chosen randomly from the k arms
            
        else: #case of exploitation
            a=np.argmax(q) #returns the action with the highest estimated expected value at current iteration
        
        reward=np.random.normal(mu[a],1) #the reward is sampled from the normal distribution with mean equalling the true expected value of arm and a std-dev of 1  
        
        #updating counts
        pulls+=1
        arm_pull_number[a]+=1
        
        #updating the rewards
        total_reward+=reward
        avg_reward[step]=total_reward/pulls
        
        
        #updating the estimated value of arm a pulled at the current iteration
        q[a]=((q[a]*(arm_pull_number[a]-1))+reward)/arm_pull_number[a]
        
    avg_reward=normalise(avg_reward) #fits the average rewards between a scale of 0 and 1
    
    #returns the estimated expected values of the arms after all the iterations and
    #the average normalised reward per iteration at each iteration
    #the number of arm pulls of each arm
    
    return q,avg_reward,arm_pull_number 

In [4]:
def ucb_1(k,iterations,mu='random'):
    
    #k=number of arms
    #iterations= number of pulls of the arms
    #mu=an array of length k, which holds the true expected value of each arm
    #mu="random"-> the expected value is sampled from a normal distribution with standard deviation 1 (default)
    #mu=user defined-> the user sends the true expected values of each arm
    
    if mu=="random":
        mu=np.random.normal(0,1,k) #now mu is an array containing the true expected values of each arm
    if len(mu)!=k: #case wwhen mu is user defined
        print("The length of the entered array of true expected values does not match with the number of arms entered \n")
        return
    
    q=np.zeros(k) #the array of the estimated expected values of each arm
    pulls=0 #number of pulls at current iteration
    arm_pull_number=np.zeros(k) #stores the number of times each arm has been pulled till the current iteration
    reward=0 #the reward earned at the current iteration
    a=0 #the arm pulled at the current iteration
    total_reward=0 #the total rewards won till the current iteration
    avg_reward=np.zeros(iterations) #the average reward per iteration till the current iteration
    
    for step in range(iterations):
        
        if step==0: #at first step we have to play each arm atleast once
            for a in range(k):
                reward=reward=np.random.normal(mu[a],1) #the reward is sampled from the normal distribution with mean equalling the true expected value of arm and a std-dev of 1
                #updating counts
                pulls+=1
                arm_pull_number[a]+=1
                #updating the rewards
                total_reward+=reward
                avg_reward[step]=total_reward/pulls
                #updating the estimated value of arm a pulled at the current iteration
                q[a]=((q[a]*(arm_pull_number[a]-1))+reward)/arm_pull_number[a]
        
        
        else:
            
            upper_bound_arm=q+np.sqrt(np.divide(2*np.log(step),arm_pull_number))
            a=np.argmax(upper_bound_arm) #choosoing the arm with maximum upper bound
        
            reward=np.random.normal(mu[a],1) #the reward is sampled from the normal distribution with mean equalling the true expected value of arm and a std-dev of 1  
        
            #updating counts
            pulls+=1
            arm_pull_number[a]+=1
        
            #updating the rewards
            total_reward+=reward
            avg_reward[step]=total_reward/pulls
        
            #updating the estimated value of arm a pulled at the current iteration
            q[a]=((q[a]*(arm_pull_number[a]-1))+reward)/arm_pull_number[a]
        
    avg_reward=normalise(avg_reward) #fits the average rewards between a scale of 0 and 1
    
    #returns the estimated expected values of the arms after all the iterations and
    #the average normalised reward per iteration at each iteration
    #the number of arm pulls of each arm
    
    return q,avg_reward,arm_pull_number 

In [5]:
def epsilon_greedy_decay(k,iterations,mu='random'):
    
    #k=number of arms
    #iterations= number of pulls of the arms
    #mu=an array of length k, which holds the true expected value of each arm
    #mu="random"-> the expected value is sampled from a normal distribution with standard deviation 1 (default)
    #mu=user defined-> the user sends the true expected values of each arm
    
    if mu=="random":
        mu=np.random.normal(0,1,k) #now mu is an array containing the true expected values of each arm
    if len(mu)!=k: #case wwhen mu is user defined
        print("The length of the entered array of true expected values does not match with the number of arms entered \n")
        return
    
    q=np.zeros(k) #the array of the estimated expected values of each arm
    pulls=0 #number of pulls at current iteration
    arm_pull_number=np.zeros(k) #stores the number of times each arm has been pulled till the current iteration
    reward=0 #the reward earned at the current iteration
    a=0 #the arm pulled at the current iteration
    total_reward=0 #the total rewards won till the current iteration
    avg_reward=np.zeros(iterations) #the average reward per iteration till the current iteration
    beta=1/k #scaling factor
    
    #decayed epsilon=1/(1+n*beta) ->n=steps taken till the current iteration
    
    for step in range(iterations):
        p=np.random.rand() #randomly generates a number between 0 and 1
        
        if p<1/(1+step*beta): #case of exploration
            a=np.random.choice(k) #action is chosen randomly from the k arms
            
        else: #case of exploitation
            a=np.argmax(q) #returns the action with the highest estimated expected value at current iteration
        
        reward=np.random.normal(mu[a],1) #the reward is sampled from the normal distribution with mean equalling the true expected value of arm and a std-dev of 1  
        
        #updating counts
        pulls+=1
        arm_pull_number[a]+=1
        
        #updating the rewards
        total_reward+=reward
        avg_reward[step]=total_reward/pulls
        
        
        #updating the estimated value of arm a pulled at the current iteration
        q[a]=((q[a]*(arm_pull_number[a]-1))+reward)/arm_pull_number[a]
        
    avg_reward=normalise(avg_reward) #fits the average rewards between a scale of 0 and 1
    
    #returns the estimated expected values of the arms after all the iterations and
    #the average normalised reward per iteration at each iteration
    #the number of arm pulls of each arm
    
    return q,avg_reward,arm_pull_number 

In [6]:
def pac(k,eps,delta,mu='random'):
    
    #k=number of arms
    #iterations= number of pulls of the arms
    #mu=an array of length k, which holds the true expected value of each arm
    #mu="random"-> the expected value is sampled from a normal distribution with standard deviation 1 (default)
    #mu=user defined-> the user sends the true expected values of each arm
    #eps=the solution is an eps optimal arm (defines the aprroximity part)
    #delta=the solution is an optimal solution with a probability of 1-delta (defines the probability part)
    
    if mu=="random":
        mu=np.random.normal(0,1,k) #now mu is an array containing the true expected values of each arm
    if len(mu)!=k: #case wwhen mu is user defined
        print("The length of the entered array of true expected values does not match with the number of arms entered \n")
        return
    
    q=np.zeros(k) #the array of the estimated expected values of each arm
    arm_pull_number=np.zeros(k) #stores the number of times each arm has been pulled till the current pull
    reward=0 #the reward earned at current pull
    a=0 #the arm pulled 

    #the number of times each arm has to be sampled
    l=round((4/(eps**2))*np.log((2*k)/delta))
    
    for a in tqdm(range(k)):
        for sample in range(l):
            reward=np.random.normal(mu[a],1) #the reward is sampled from the normal distribution with mean equalling the true expected value of arm and a std-dev of 1  
            
            #updating counts
            arm_pull_number[a]+=1
            
            #updating the estimated value of arm a pulled at the current iteration
            q[a]=((q[a]*(arm_pull_number[a]-1))+reward)/arm_pull_number[a]
        
            
    optimal_arm=np.argmax(q)        
    
    #returns the epsilon optimal arm with a prob of (1-delta) 
    #returns the estimated expected values of the arms after all the pulls 
    #the number of samples drawn for each arm
    
    return optimal_arm,q,l

In [7]:
k=10 #number of arms
mu=np.random.normal(0,1,k) #true expected rewards of each arm
optimal_arm01,q01,samples_drawn01=pac(k,0.1,0.1,mu)
optimal_arm05,q05,samples_drawn05=pac(k,0.01,0.1,mu)
optimal_arm09,q09,samples_drawn09=pac(k,0.001,0.1,mu)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # This is added back by InteractiveShellApp.init_path()
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [15:20<00:00, 92.01s/it]


<h1> Quantitative Analysis - PAC BOUNDS </h1>

In [8]:
print("the true expected rewards for each arm: ")
print(mu)
print("\n")
a=np.argmax(mu)
print("the index of the true optimal arm: ",a)

the true expected rewards for each arm: 
[ 1.70834451 -0.95313593 -0.91389804 -0.290868    0.37686599  0.25138032
  0.10793535  1.73948035  0.50503734  1.41603776]


the index of the true optimal arm:  7


<h3> Case 1: epsilon=0.1 and delta=0.1 </h3>

In [9]:
print("the estimated expected rewards for each arm: ")
print(q01)
print("\n")
print("the index of the estimated optimal arm: ",optimal_arm01)
print("\n")
print("the difference between the true and the estimated expected reward of the optimal arm: ")
print(abs(q01[a]-mu[a]))
print("\n")
print("the number of pulls/samples drawn for each of the ",k," arms: ",samples_drawn01)

the estimated expected rewards for each arm: 
[ 1.69726299 -0.91194481 -0.92310409 -0.30226664  0.35582659  0.22545138
  0.0957777   1.7596432   0.4894812   1.42753234]


the index of the estimated optimal arm:  7


the difference between the true and the estimated expected reward of the optimal arm: 
0.02016284525833356


the number of pulls/samples drawn for each of the  10  arms:  2119


<h3> Case 2: epsilon=0.01 and delta=0.1 </h3>

In [10]:
print("the estimated expected rewards for each arm: ")
print(q05)
print("\n")
print("the index of the estimated optimal arm: ",optimal_arm05)
print("\n")
print("the difference between the true and the estimated expected reward of the optimal arm: ")
print(abs(q05[a]-mu[a]))
print("\n")
print("the number of pulls/samples drawn for each of the ",k," arms: ",samples_drawn05)

the estimated expected rewards for each arm: 
[ 1.71177688 -0.94839804 -0.91648705 -0.29049531  0.38312312  0.25185082
  0.11144743  1.73830878  0.50637544  1.41454997]


the index of the estimated optimal arm:  7


the difference between the true and the estimated expected reward of the optimal arm: 
0.001171571410780059


the number of pulls/samples drawn for each of the  10  arms:  211933


<h3> Case 3: epsilon=0.001 and delta=0.1 </h3>

In [11]:
print("the estimated expected rewards for each arm: ")
print(q09)
print("\n")
print("the index of the estimated optimal arm: ",optimal_arm09)
print("\n")
print("the difference between the true and the estimated expected reward of the optimal arm: ")
print(abs(q09[a]-mu[a]))
print("\n")
print("the number of pulls/samples drawn for each of the ",k," arms: ",samples_drawn09)

the estimated expected rewards for each arm: 
[ 1.70855173 -0.95298418 -0.91368171 -0.2911788   0.3767244   0.25122893
  0.10807299  1.73954543  0.50487605  1.41608851]


the index of the estimated optimal arm:  7


the difference between the true and the estimated expected reward of the optimal arm: 
6.507669860456033e-05


the number of pulls/samples drawn for each of the  10  arms:  21193269


-Comparison is done on the basis of different values of epsilon keeping the probability of finding the optimal arm (1-delta)
the same for all three cases

-the true optimal arm was returned for all eps=0.1, 0.01 and 0.001

-the number of samples drawn for each arm for the respective eps=0.1,0.01 and 0.001 are 2119, 211933 and 21193269

-the estimated expected rewards were the closest to the true expected rewards for eps=0.001 